# Putting Together Webscraping HTMLs and parsing through the page
by Harsha and Jesse

In [8]:
# BeautifulSoup and requests library import
from bs4 import BeautifulSoup
import requests 
import re
import pandas as pd
import time

In [9]:
#cleaned htmls (entire html should work)
#use hte "final_topic".csv for this part
htmls = pd.read_csv("final_politics.csv", index_col = 0, header = None).reset_index(drop=True)

In [10]:
#create a list from the dataframe with htmls
h = htmls[1].to_list()

In [11]:
#create a dictionary to fill with keys: urls, data: data from article
htmls_with_data = {}

htmls_with_data

{}

In [ ]:
#making for loop to scrape the data:
counter = 1

for i in h:
    time.sleep(5)
    #page request
    page_response = requests.get(i, timeout=5)
    print(page_response)
    
    #parsing content from page and storing it
    page_content = BeautifulSoup(page_response.content, "html.parser")
    
    #empty list to store user data
    data = []
    
    #try to grab header
    try:
        #grabs div that stores title info
        titleInfo = page_content.find_all('div',attrs={"class":"n p"})

        #finds title within titleInfo div
        href = str(titleInfo).index("<h1")
        start = str(titleInfo)[href:].index(">")+href+1
        end = str(titleInfo)[href:].index("</h1>")+href

        #appends title to data list after removing extra tags
        title = re.sub(re.compile('<.*?>'), '', str(titleInfo)[start:end])
        data.append(title)
    except:
        data.append(None)
    #try to grab author
    try:

        #<h1>Finding and Adding Author to Data</h1>

        #grabs div that stores user info
        userInfo = page_content.find_all('div',attrs={"class":"o n"})

        #finds authors name within userInfo div
        href = str(userInfo)[24:].index("<a class=")+24
        start = str(userInfo)[href:].index(">")+href+1
        end = str(userInfo)[href:].index("</a>")+href

        #appends authors name to data list
        author = str(userInfo)[start:end]
        data.append(author)
    except:
        data.append(None)
    try:
        #trying to get author page url:

        #<h1>Finding and Adding Author Page URL to Data</h1>

        #finds date within userInfo div
        href = str(userInfo)[24:].index("<a class=")
        start = str(userInfo)[href:].index("href=")+href+6
        end = str(userInfo)[start:].index("?source")+start

        #appends date to data list
        url = "medium.com" + str(userInfo)[start:end]
        data.append(url)
    except:
        data.append(None)
        
    ### PARSING THROUGH USE PAGE NOW ###
    try: #adding user data:
        #user url
        user_link = "https://" + data[2]
        #user page request
        time.sleep(5)
        user_response = requests.get(user_link, timeout=5)
        #parsing content from user page to store follower count
        user_content = BeautifulSoup(user_response.content, "html.parser")
        #user tag
        user_href = re.sub("medium.com", "", data[2])
    except:
        pass
    
    try: #get member since
        memberInfo = user_content.find_all("span")
        year = re.findall("[0-9]{4}", str(memberInfo))[0]
        data.append(year)
    except:
        data.append(None)
    
    try: #get following:
        #returns following count and adds to data list
        followingInfo = user_content.find_all('a', attrs={"href":user_href+"/following"})
        following = re.sub("[^0-9]", "", str(followingInfo))
        data.append(following)
    except:
        data.append(None)
        
    #get follower count
    try: 
        #returns follower count and adds to data list
        followerInfo = user_content.find_all('a', attrs={"href":user_href+"/followers"})
        followers = re.sub("[^0-9]", "", str(followerInfo))
        data.append(followers)
    except:
        data.append(None)
        
    #trying to add date
    try:
        #<h1>Finding and Adding Date to Data</h1>

        #finds date within userInfo div
        href = str(userInfo)[end:].index("<a class=")+end
        start = str(userInfo)[href:].index(">")+href+1
        end = str(userInfo)[href:].index("</a>")+href

        #appends date to data list
        date = str(userInfo)[start:end]
        data.append(date)
    except:
        data.append(None)

    #tring to add clap number
    try:
        #<h1>Finding and Adding Number of Claps to Data</h1>

        #grabs all paragraphs in the article
        clapInfo = page_content.find_all('div',attrs={"class":"n o"})

        #finds claps within clapInfo div
        result = re.sub(re.compile('<.*?>'), '', str(clapInfo))
        result = re.findall("\d+", result)

        #appends claps to data list
        if len(result) == 0:
            data.append('0')

        else:
            data.append(result[0])
    except:
        data.append(None)
        
    #getting all the text data:
    try:
        
        #<h1>Adding a List of the Text Content to Data</h1>

        #grabs all paragraphs in the article
        textContent = page_content.find_all("p")
        textInfo = str(textContent)

        #RegEx that cleans paragraph data by removing html tags and extra commas
        result = re.sub(re.compile('<.*?>'), '', textInfo)
        result = re.split("\., ", result)
        result = result[:-1]
        data.append(result)
    except:
        data.append(None)
    
    #append html to dictionary with data
    htmls_with_data[i] = data

    
    #counter
    print(counter)
    counter = counter + 1

<Response [200]>
1
<Response [200]>
2
<Response [200]>
3
<Response [200]>
4
<Response [200]>
5
<Response [200]>
6
<Response [200]>
7
<Response [200]>
8
<Response [200]>
9
<Response [200]>
10
<Response [200]>
11
<Response [200]>
12
<Response [200]>
13
<Response [200]>
14
<Response [200]>
15
<Response [200]>
16
<Response [200]>
17
<Response [200]>
18
<Response [200]>
19
<Response [200]>
20
<Response [200]>
21
<Response [200]>
22
<Response [200]>
23
<Response [200]>
24
<Response [200]>
25
<Response [200]>
26
<Response [200]>
27
<Response [200]>
28
<Response [200]>


In [86]:
#turn dictionary into a df
dictionary_of_data = pd.DataFrame(htmls_with_data)

In [89]:
dictionary_of_data.shape

(6, 519)

In [71]:
#pivot to make the index the url and resetting to get it
full_data = dictionary_of_data.T.reset_index()

In [73]:
#change the urls to be something easier to interpret
full_data.columns = ["url", "title", "author", "username", "published", "claps", "text"]

In [75]:
#to csv
full_data.to_csv("Medium_Politics_Data.csv")